In [1]:
%load_ext cython

In [2]:
%%cython -a

cdef class Record(object):
    
    """
    A record is basically data attached to a bounding box.
    
    Bounding boxes work the same way that GeoJSON assumes: (minx, miny, maxx, maxy)
    
    Parameters:
        identifier - an unsigned integer to identify the record
        bounds - the bounding box for the record
    """
    
    cdef:
        double bounds[4]
        unsigned int identifier
    
    cdef inline bint overlaps(self, double *bounds):
        "Whether the record contains data in the bounding box"
        return (
            self.bounds[0] < bounds[2] 
            and self.bounds[2] > bounds[0]
            and self.bounds[1] < bounds[3]
            and self.bounds[3] > bounds[1]
        )

cdef class Node(object):
    
    """
    On the first day of Christmas, my true love gave to me,
    ... a no-o-ode in an RTree. 🎵
    
    Parameters:
    """
    
    cdef:
        Node parent
        list records
        bint is_leaf
    
    def __cinit__(self, Node parent):
        self.parent = parent
        self.records = list()

cdef class RTree(object):
    
    """
    An index over two-dimensional items
    
    Parameters:
        min_items, max_items - the minimum and maximum 
            number of items that a leaf node can hold. For COGJ
            this will be the size of the FeatureCollections
            that are served.
    """
    
    cdef:
        Node root
        unsigned int max_level  # index of leaf nodes
        unsigned int count      # total number of nodes
        unsigned int min_items, max_items
        
    def __cinit__(self):
        self.root = Node(None)
        self.root.is_leaf = True
        self.max_level = 0
        self.count = 0
    
    def __init__(self, min_items=5, max_items=50):
        self.min_items = min_items
        self.max_items = max_items
    
    def __len__(self):
        return self.count
    
    def insert(self, object, object coords):
        """
        Insert an object into the RTree
        """
        if not isinstance(coords, (list, tuple)):
            raise TypeError('Coordinates as a list or tuple expected')